1.a. what is supervised finetuning?
when we train a pretrained model on a new task to become in handy for a specific task we call it finetuning. when it is trained over a labeled dataset
b. These models have been trained on very large datasets, billions of tokens, but they are not capable of doing sth extra, for example pure pretrained base model cant write TV script or answer to the questions and ...
c. supervised finetuned models are capable of doing tasks like translation answering to questions writing short stories but base models (next token prediction) can't understand the prompts and do not do them accurately.






2. Parameter efficient fine tuning means we can make the model capable of doing a task without updating and training all of the parameters. For example, we can fine tune a LLM on a specific task like question and answering dataset to be able to answer people's questions. Like chat GPT!
we have LoRA, adapters, prompt tuning, prefix tunning.
LoRA: instead of fine tuning the whole model characters we add extra parameters to the model's parameters and fine tune them
Adapters: add a small NN between the layers of a pretrained model
Prompt tuning: instead of trainig the whole model it learns some special tokens
Prefix Tuning: learns a small number of tokens, but instead of sending them to the input, it injects them into the key and value vectors.

peft is usefule cause it is memory efficient, easier and faster for training also the most important part is "it is safe" when data sets are small and the model is large without using peft methods we will face overfitting and the model will lose the NL acilities.

3. a.Dataset Selection and Preparation:
I chose lmsys/chatbot_arena_conversations dataset which is a labaled dataset for trainig chat bots!
b.it is pares of question and answers between a human and a chat bot!but in reality it consists of conversations and question answers between different AIs
we enter the tokens like ###human ### assisstant to the modeland tokenizer will split it into small tokens
c. the dataset has pares of questions ans answers I sent each pares at the same time to the model also I labeled the data so the model will know which part is a question by a human and which part is the related answer since I am injecting both question and answer at the same time it will garantee that the model learns it well.


4. For this assignment I chose GPT2 medium it has around 343M character and I decided to do the fine tuning with LoRA for the reasons I mentioned above.
b. For applying LoRA I used PEFT library with the following parameters:
    r=8,it is small rnough to be able to run and also it is a low rank matrix of characters.
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["c_attn"]  



In [3]:
%%capture

!pip install transformers datasets peft accelerate bitsandbytes trl --upgrade


In [1]:
%%capture

!pip install datasets

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `n` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when p

In [25]:
from datasets import load_dataset, DatasetDict

# Step 1: Load and shuffle
dataset = load_dataset("lmsys/chatbot_arena_conversations", split="train")
dataset = dataset.shuffle(seed=42)

# Step 2: Define extraction function
def extract_one_turn(example):
    conv = example.get("conversation_a", [])

    # We only take the first valid human→assistant pair
    if len(conv) >= 2:
        first = conv[0]
        second = conv[1]

        if first["role"] == "user" and second["role"] == "assistant":
            prompt = first["content"].strip()
            response = second["content"].strip()

            # Optional filters: remove toxic, strange or empty prompts
            if prompt and response and 10 < len(prompt) < 1000 and 10 < len(response) < 1500:
                text = f"### Human:\n{prompt}\n\n### Assistant:\n{response}"
                return {"text": text}

    return {"text": None}

# Step 3: Apply to full dataset
clean_dataset = dataset.map(extract_one_turn)
clean_dataset = clean_dataset.filter(lambda x: x["text"] is not None)

# Step 4: Split into train/val/test
split_dataset = clean_dataset.train_test_split(test_size=0.2, seed=42)
test_valid = split_dataset['test'].train_test_split(test_size=0.5, seed=42)

final_dataset = DatasetDict({
    "train": split_dataset["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"]
})


Map:   0%|          | 0/33000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/33000 [00:00<?, ? examples/s]

In [26]:
print(final_dataset["train"][0]["text"])


### Human:
What is a black hole?

### Assistant:
A black hole is a region of spacetime exhibiting such strong gravitational effects that nothing—not even light—can escape from inside it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to create a black hole. Around a black hole there is a mathematically defined surface called an event horizon that marks the point of no return. It is called "black" because it absorbs all the light that hits the horizon, reflecting nothing, just like a perfect black body in thermodynamics. Quantum field theory in curved spacetime predicts that event horizons emit radiation like a black body with a finite temperature. This temperature is inversely proportional to the mass of the black hole, making it difficult to observe this predicted radiation for black holes of stellar mass or greater.


In [27]:
from transformers import AutoTokenizer

model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Important for GPT-2


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token


In [29]:
tokenized_dataset = final_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/20913 [00:00<?, ? examples/s]

Map:   0%|          | 0/2614 [00:00<?, ? examples/s]

Map:   0%|          | 0/2615 [00:00<?, ? examples/s]

In [30]:
print(tokenized_dataset["train"][0]["input_ids"][:20])
print(tokenizer.decode(tokenized_dataset["train"][0]["input_ids"]))


[21017, 5524, 25, 198, 2061, 318, 257, 2042, 7604, 30, 198, 198, 21017, 15286, 25, 198, 32, 2042, 7604, 318]
### Human:
What is a black hole?

### Assistant:
A black hole is a region of spacetime exhibiting such strong gravitational effects that nothing—not even light—can escape from inside it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to create a black hole. Around a black hole there is a mathematically defined surface called an event horizon that marks the point of no return. It is called "black" because it absorbs all the light that hits the horizon, reflecting nothing, just like a perfect black body in thermodynamics. Quantum field theory in curved spacetime predicts that event horizons emit radiation like a black body with a finite temperature. This temperature is inversely proportional to the mass of the black hole, making it difficult to observe this predicted radiation for black holes of stellar mass or greater.<|endoftext|>

In [32]:
def tokenize_and_set_labels(example):
    tokenized = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = final_dataset.map(
    tokenize_and_set_labels,
    batched=True,
    remove_columns=["text"]  # remove if you don’t need the raw text anymore
)


Map:   0%|          | 0/20913 [00:00<?, ? examples/s]

Map:   0%|          | 0/2614 [00:00<?, ? examples/s]

Map:   0%|          | 0/2615 [00:00<?, ? examples/s]

In [33]:
!pip install transformers peft datasets accelerate bitsandbytes


In [34]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

base_model = AutoModelForCausalLM.from_pretrained(
    "gpt2-medium",
    device_map="auto",  # Auto-detect GPU
    torch_dtype="auto"  # Use float16 if GPU allows
)

# Define LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["c_attn"]  # GPT-2 uses 'c_attn' for self-attention projection
)

# Wrap model with LoRA
model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 786,432 || all params: 355,609,600 || trainable%: 0.2212


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  return output_tensor


In [35]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT-style training is autoregressive, not masked
)


In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-medium-lora-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=True,  # Only if Colab supports it
    report_to="none",
    logging_steps=50
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [37]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


<ipython-input-37-4e961172ac27>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.099000,1.989626
2,2.291300,1.966894
3,2.065600,1.961880


TrainOutput(global_step=15687, training_loss=2.1047601301628887, metrics={'train_runtime': 1697.6529, 'train_samples_per_second': 36.956, 'train_steps_per_second': 9.24, 'total_flos': 5.841732522265805e+16, 'train_loss': 2.1047601301628887, 'epoch': 3.0})

In [38]:
model.save_pretrained("./gpt2-medium-lora-finetuned")
tokenizer.save_pretrained("./gpt2-medium-lora-finetuned")


('./gpt2-medium-lora-finetuned/tokenizer_config.json',
 './gpt2-medium-lora-finetuned/special_tokens_map.json',
 './gpt2-medium-lora-finetuned/vocab.json',
 './gpt2-medium-lora-finetuned/merges.txt',
 './gpt2-medium-lora-finetuned/added_tokens.json',
 './gpt2-medium-lora-finetuned/tokenizer.json')

In [39]:
test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])


In [40]:
import math

perplexity = math.exp(test_results["eval_loss"])
print(f"Test Perplexity: {perplexity:.2f}")


Test Perplexity: 7.22


In [41]:
# Evaluate on test split
test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])

# Calculate perplexity
import math
perplexity = math.exp(test_results["eval_loss"])

# Display results
print(f"Test Loss: {test_results['eval_loss']:.4f}")
print(f"Test Perplexity: {perplexity:.2f}")


Test Loss: 1.9775
Test Perplexity: 7.22


In [42]:
input_text = "### Human:\nWhat's the capital of Italy?\n\n### Assistant:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Human:
What's the capital of Italy?

### Assistant:
The capital of Italy is Rome, Italy. It is the capital of the Italian state and is the largest city in the country. It is also the largest city in the world, with a population of over 1.5 billion people. Rome is


In [43]:
input_text = "### Human:\nWhy sky is blue?\n\n### Assistant:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Human:
Why sky is blue?

### Assistant:
Sky is blue because it is the color of the sky. It is the color of the sky because it is the color of the sky because it is the color of the sky because it is the color of the sky because it is the color of


In [44]:
input_text = "### Human:\nwho are you?\n\n### Assistant:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Human:
who are you?

### Assistant:
I am an AI language model that can answer questions about the human language. I can answer questions about the human language in a variety of ways, including:

- What is the meaning of a word?
- What is the meaning of


In [45]:
input_text = "### Human:\n do we have a blackhole in solar system?\n\n### Assistant:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Human:
 do we have a blackhole in solar system?

### Assistant:
Yes, there is a blackhole in the solar system. It is located in the constellation of Cygnus, and is about 1.5 times the mass of the sun. It is about 1.5 times the size of the sun,


How GPT2 medium works before fine tuning?

In [46]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the base GPT-2 Medium model (before fine-tuning)
base_model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

# Load the fine-tuned model with LoRA
from peft import PeftModel
fine_tuned = PeftModel.from_pretrained(base_model, "./gpt2-medium-lora-finetuned")
fine_tuned.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1024)
        (wpe): Embedding(1024, 1024)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2Block(
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=3072, nx=1024)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
     

In [58]:
prompt = "### Human:\n can you explain what DeepSeek-V2 is and how it improves over DeepSeek-V1?\n\n### Assistant:"
inputs = tokenizer(prompt, return_tensors="pt").to(fine_tuned.device)


In [59]:
# Base model (before fine-tuning)
with torch.no_grad():
    base_outputs = base_model.generate(**inputs, max_new_tokens=50)
    base_text = tokenizer.decode(base_outputs[0], skip_special_tokens=True)

# Fine-tuned model (after LoRA)
with torch.no_grad():
    lora_outputs = fine_tuned.generate(**inputs, max_new_tokens=50)
    lora_text = tokenizer.decode(lora_outputs[0], skip_special_tokens=True)

print(" Base GPT-2 Medium Output:")
print(base_text)
print(" Fine-Tuned GPT-2 Medium with LoRA Output:")
print(lora_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Base GPT-2 Medium Output:
### Human:
 can you explain what DeepSeek-V2 is and how it improves over DeepSeek-V1?

### Assistant:
DeepSeek-V2 is a new version of DeepSeek that improves on the performance and features of DeepSeek-V1. DeepSeek-V2 is designed to be a more efficient and reliable deep learning framework that can
 Fine-Tuned GPT-2 Medium with LoRA Output:
### Human:
 can you explain what DeepSeek-V2 is and how it improves over DeepSeek-V1?

### Assistant:
DeepSeek-V2 is a new version of DeepSeek that improves on the performance and features of DeepSeek-V1. DeepSeek-V2 is designed to be a more efficient and reliable deep learning framework that can
